## Using K-Means to compare neighborhoods between Manhattan and Toronto

### 1. Introduction

> Juan Pedro is a 25 year old statistician living in the neighbourhood of St. James Town in Toronto, Canada. Juan has a particular affinity to his current neighbourhood because of the eclectic mix of restaurants, bars, cafes and other venues in his immediate vicinity. For personal reasons however, Juan is required to relocate to Manhattan, New York. He is therefore seeking a shortlist of suitable neighbourhoods in Manhattan to which to relocate which are similar to St. James Town in terms of the local mix of venues. 

### 2. Data Methodology

 > To provide a suitable recommendation to Juan Pedro on the neighbourhoods in Manhattan which are similar to St. James Town, all neighbourhoods in both Toronto and Manhattan will be clustered using k-means clustering. The clustering will be based on the composition of the top 10 venues within a 500m radius of the geographic center of each neighbourhood. Those neighbourhoods in Manhattan which are also in the same cluster as St. James Town will then comprise the shortlist of neighbourhoods that John should consider relocating to. 

> The FourSquare.com database has been queried previously and the top 10 venues within 500m of each neighbourhood in Manhattan have been identified. The latitude and longitude of each venue and each neighbourhood have been identified. 

## 3. Method (K-Means)

To conduct the analysis the dataframes developed in previous assignments will be merged into a single dataframe and clustered using k-means clustering, based upon the mix of top 10 venues within 500m of each neighbourhood. The Cluster containing the neighbourhood of St. James Town, Toronto will then be identified. Neighbourhoods in Manhattan also included in the same cluster will then form the shortlist of recommended neighbourhoods for John Smith to consider for relocation.  

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 3.1 Useful functions (Def)

In [2]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## 4. Manhattan's data

In [4]:
with open('C:/Users/elton.junior/Downloads/nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)


for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

CLIENT_ID = 'JTJG0JSV3AX5DOIOMYGJJYWRSOY0IWCVLHJKH32N3QVC4UU0' # your Foursquare ID
CLIENT_SECRET = 'XLNPRZEDPL2MMXTAVDQZVOEM5L2MLTOFIIZPEKWD1U2I0MJF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


manhattan_venues['City'] = 'Manhattan'

print(manhattan_venues.shape)
manhattan_venues.head()

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
(3093, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [5]:
manhattan_df=manhattan_data.rename(columns = {'Neighborhood':'Neighbourhood', 'Neighborhood Latitude': 'Neighbourhood Latitude', 'Neighborhood Longitude':'Neighbourhood Longitude'})
manhattan_df['City']= 'Manhattan'
col=manhattan_df.columns.tolist()
col=col[-1:] + col[:-1]
manhattan_df=manhattan_df[col]
manhattan_df.head()

,City,Borough,Neighbourhood,Latitude,Longitude
0,Manhattan,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Manhattan,Hamilton Heights,40.823604,-73.949688


## 5. Toronto's data

In [6]:
toronto = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

toronto = toronto.loc[toronto.Borough != 'Not assigned'] # Removing 'Not Assigned' Boroughs

lat_long = pd.read_csv('http://cocl.us/Geospatial_data') # Provided link to lat/long data
toronto = pd.merge(toronto, lat_long, on = 'Postal Code')

address = 'Toronto City, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

toronto_venues['City'] = 'Toronto'

print(toronto_venues.shape)
toronto_venues.head()

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [23]:
#Add City borough of Toronto to this dataset and rearrange columns

toronto_df=toronto.drop('Postal Code', axis = 1)
toronto_df['City']= 'Toronto'
cols=toronto_df.columns.tolist()
cols=cols[-1:] + cols[:-1]
toronto_df=toronto_df[cols]
toronto_df.rename(columns= {'Neighborhood':'Neighbourhood'}, inplace = True)

toronto_df.head()

,City,Borough,Neighbourhood,Latitude,Longitude
0,Toronto,North York,Parkwoods,43.753259,-79.329656
1,Toronto,North York,Victoria Village,43.725882,-79.315572
2,Toronto,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,Toronto,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Toronto,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 6. Combining datasets

In [25]:
#Combine datasets for Manhattan and Toronto into a single dataframe and create a second dataframe without venue information, removing duplicate entries for neighbourhoods. 
combined_data=pd.concat([manhattan_df, toronto_df])
combined_data.drop_duplicates(subset=None, keep='first', inplace=True)
combined_data.head()

,City,Borough,Neighbourhood,Latitude,Longitude
0,Manhattan,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Manhattan,Hamilton Heights,40.823604,-73.949688


In [30]:
#Version of the Combined Data without Venue information
combined_df =pd.concat([manhattan_venues, toronto_venues])
combined_df.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place,Manhattan
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio,Manhattan
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner,Manhattan
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop,Manhattan
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop,Manhattan


In [35]:
#Group by venue count in each neighbourhood of each city
combined_df.groupby('Neighborhood', as_index=False).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City
0,Agincourt,4,4,4,4,4,4,4
1,"Alderwood, Long Branch",11,11,11,11,11,11,11
2,"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21,21
3,Battery Park City,71,71,71,71,71,71,71
4,Bayview Village,4,4,4,4,4,4,4
5,"Bedford Park, Lawrence Manor East",23,23,23,23,23,23,23
6,Berczy Park,58,58,58,58,58,58,58
7,"Birch Cliff, Cliffside West",4,4,4,4,4,4,4
8,"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23,23
9,"Business reply mail Processing Centre, South C...",17,17,17,17,17,17,17


In [37]:
#Use One-hot encoding to identify which types of venues are present in each Neighbourhood
combined_onehot = pd.get_dummies(combined_df[['Venue Category']], prefix="", prefix_sep="")
combined_onehot['Neighborhood'] = combined_df['Neighborhood'] 
fixed_columns = [combined_onehot.columns[-1]] + list(combined_onehot.columns[:-1])
combined_onehot = combined_onehot[fixed_columns]
combined_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,O

In [38]:
#Find the mean of the number of venue types in each Neighbourhood
combined_grouped = combined_onehot.groupby('Neighborhood').mean().reset_index()
combined_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,O

In [39]:
#Define a function to return most common venues from a dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [84]:
#Place top 10 venues in each Neighbourhood in a new dataframe:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighborhood'] = combined_grouped['Neighborhood']

for ind in np.arange(combined_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(combined_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Skating Rink,Lounge,Women's Store,Egyptian Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Athletics & Sports,Dance Studio,Pharmacy,Pub,Pool,Sandwich Place,Coffee Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Ice Cream Shop,Sandwich Place,Diner,Fried Chicken Joint,Deli / Bodega,Sushi Restaurant,Restaurant,Supermarket
3,Battery Park City,Park,Coffee Shop,Hotel,Boat or Ferry,Memorial Site,Gym,Gourmet Shop,Shopping Mall,Mexican Restaurant,Food Court
4,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store,Doctor's Office,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner


## 7. Clustering

In [86]:
#Use K-Means to cluster Neighbourhoods into 8 Clusters, based on the types of venues in each Neighbourhood
#Using a lower value for k does not provide sufficient differentiation of neighbourhoods in Manhattan

kclusters = 8
combined_grouped_clustering = combined_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(combined_grouped_clustering)
kmeans.labels_[0:10] 

array([2, 2, 5, 2, 5, 2, 2, 5, 2, 2])

In [87]:
# add clustering labels and merge to dataframe without venue info (combined_data) data to add longitude and latitude for each Neighbourhood
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
combined_merge = combined_data.merge(neighbourhoods_venues_sorted, right_on='Neighborhood', left_on = 'Neighbourhood')
combined_merge.head()

,City,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Manhattan,Marble Hill,40.876551,-73.910660,2,Marble Hill,Sandwich Place,Gym,Coffee Shop,Steakhouse,Bank,Kids Store,Discount Store,Diner,Donut Shop,Tennis Stadium
1,Manhattan,Manhattan,Chinatown,40.715618,-73.994279,2,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,American Restaurant,Salon / Barbershop,Spa,Ice Cream Shop,Optical Shop,Vietnamese Restaurant
2,Manhattan,Manhattan,Washington Heights,40.851903,-73.936900,2,Washington Heights,Café,Bakery,Deli / Bodega,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Gym,Supermarket,Supplement Shop,Sandwich Place
3,Manhattan,Manhattan,Inwood,40.867684,-73.921210,2,Inwood,Mexican Restaurant,Pizza Place,Lounge,Café,Restaurant,Spanish Restaurant,Chinese Restaurant,Park,Deli / Bodega,American Restaurant
4,Manhattan,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Hamilton Heights,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Bakery,Indian Restaurant,Cocktail Bar,Chinese Restaurant,Caribbean Restaurant


In [88]:
#Find the Cluster in which St. James Town, Toronto is located

df_lookup=combined_merge.set_index('Neighbourhood')
print('St. James Town is in Cluster Category ', df_lookup.loc['St. James Town', 'Cluster Labels'])

St. James Town is in Cluster Category  2


## 8. Clustering Analysis

Other Neighbourhoods in Cluster 2 are listed in the table below

In [83]:
combined_merge.loc[combined_merge['Cluster Labels'] == 5, combined_merge.columns[[0] + [1] + list(range(5, combined_merge.shape[1]))]]

,City,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Manhattan,5,Marble Hill,Sandwich Place,Gym,Coffee Shop,Steakhouse,Bank,Kids Store,Discount Store,Diner,Donut Shop,Tennis Stadium
1,Manhattan,Manhattan,5,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,American Restaurant,Salon / Barbershop,Spa,Ice Cream Shop,Optical Shop,Vietnamese Restaurant
2,Manhattan,Manhattan,5,Washington Heights,Café,Bakery,Deli / Bodega,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Gym,Supermarket,Supplement Shop,Sandwich Place
3,Manhattan,Manhattan,5,Inwood,Mexican Restaurant,Pizza Place,Lounge,Café,Restaurant,Spanish Restaurant,Chinese Restaurant,Park,Deli / Bodega,American Restaurant
4,Manhattan,Manhattan,5,Hamilton Heights,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Bakery,Indian Restaurant,Cocktail Bar,Chinese Restaurant,Caribbean Restaurant
5,Manhattan,Manhattan,5,Manhattanville,Coffee Shop,Seafood Restaurant,Italian Restaurant,Mexican Restaurant,Park,Chinese Restaurant,Fried Chicken Joint,Sushi Restaurant,BBQ Joint,Gastropub
6,Manhattan,Manhattan,5,Central Harlem,African Restaurant,Cosmetics Shop,Bar,French Restaurant,Chinese Restaurant,American Restaurant,Seafood Restaurant,Event Space,Beer Bar,Fried Chicken Joint
7,Manhattan,Manhattan,5,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Thai Restaurant,Latin American Restaurant,Dance Studio,Gas Station,Doctor's Office,French Restaurant,Taco Place
8,Manhattan,Manhattan,5,Upper East Side,Italian Restaurant,Gym / Fitness Center,Coffee Shop,Bakery,Juice Bar,Spa,French Restaurant,Exhibit,Yoga Studio,Wine Shop
9,Manhattan,Manhattan,5,Yorkville,Coffee Shop,Gym,Italian Restaurant,Bar,Deli / Bodega,Sushi Restaurant,Wine Shop,Pizza Place,Japanese Restaurant,Diner


## 9. Diagnosis

39 possible neighbourhoods in Manhattan have been identified as being in the same cluster as St. James Town, in Toronto. Some of these are:

- Marble Hill
- Flatiron
- Tudor City
- Financial District
- Hudson Yards

The clusters are identified on the two maps below

In [89]:
#Set up Nominatim on Toronto, Canada 
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [94]:
%matplotlib inline
print('Map of Clusters in Toronto')

#Create Map of Clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(combined_merge['Latitude'], combined_merge['Longitude'], combined_merge['Neighbourhood'], combined_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

Map of Clusters in Toronto


In [95]:
#Set up Nominatim on Manhattan 
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [96]:
print('Map of Clusters in Manhattan')

#Create Map of Clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(combined_merge['Latitude'], combined_merge['Longitude'], combined_merge['Neighbourhood'], combined_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

Map of Clusters in Manhattan


## 10. Discussion

The above clustering of neighbourhoods in both Toronto and Manhattan successfully segments the neighbourhoods according to the mix of top 10 venues within a 500m radius of each neighbourhood. It is notable however, that several of the clusters contain multiple neighbourhoods and a lower value of k would result in too few clusters to make a meaningful recommendation.   

## 11. Conclusion

K-means clustering is an appropriate method of segmenting geographical neighbourhoods according to some distinct feature of each neighbourhood. In this example we have been able to make a list of 39 neighbourhoods in Manhattan which are similar to Juan's current location of St. James Town, Toronto. Thus, Juan is able to confidently limit his search for a new home to the  areas listed in the Results Section. 